In [10]:
import pandas as pd

# Load data with timestamps
df = pd.read_csv(r'C:\Users\ayedr\week-5-credit-risk-model\data\processed\cleaned_data.csv')
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Calculate RFM
snapshot_date = df['TransactionStartTime'].max()
rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,
    'TransactionId': 'count',
    'Amount': 'sum'
}).rename(columns={
    'TransactionStartTime': 'Recency',
    'TransactionId': 'Frequency',
    'Amount': 'Monetary'
}).reset_index()
print(rfm.head())  # Show first 5 rows of RFM data

        CustomerId  Recency  Frequency  Monetary
0     CustomerId_1       83          1  -10000.0
1    CustomerId_10       83          1  -10000.0
2  CustomerId_1001       89          5   20000.0
3  CustomerId_1002       25         11    4225.0
4  CustomerId_1003       11          6   20000.0


In [11]:
print(df[df['Amount'] < 0]['FraudResult'].value_counts(normalize=True))

FraudResult
0    0.999974
1    0.000026
Name: proportion, dtype: float64


In [12]:
#Investigate Outliers
#Check if extreme negative values are errors:
print(df[df['Amount'] < 0]['Amount'].describe())

count     38183.000000
mean      -3817.251408
std        9707.638935
min     -200000.000000
25%       -3000.000000
50%        -100.000000
75%         -50.000000
max          -1.200000
Name: Amount, dtype: float64


In [13]:
#Clip or remove implausible values (e.g., ≤ -$100,000):

df = df[df['Amount'] > -100000]  # Keep transactions > -$100K

In [14]:
#Standardize features to minimize skew from negatives:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

In [16]:
#Perform K-Means Clustering on RFM

# Get cluster centers (in original scale after inverse transform)
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=['Recency', 'Frequency', 'Monetary']
)
print("Cluster Centers (Original Scale):")
print(cluster_centers)

Cluster Centers (Original Scale):
     Recency    Frequency      Monetary
0  11.634783    34.849565  1.254955e+05
1  28.000000  4091.000000 -1.049000e+08
2  60.825598     7.839662  5.207238e+04


In [17]:
#Filter out customers in Cluster 1 before labeling:

rfm = rfm[rfm['Cluster'] != 1]  # Drop Cluster 1

In [18]:
#Reassign Cluster Labels
rfm['Cluster'] = rfm['Cluster'].map({0: 0, 2: 1})  # Rename clusters
rfm['is_high_risk'] = rfm['Cluster']  # Directly use as risk label

In [19]:
#Validate Distribution

print("High-Risk Distribution:")
print(rfm['is_high_risk'].value_counts(normalize=True))

High-Risk Distribution:
is_high_risk
0    0.617947
1    0.382053
Name: proportion, dtype: float64


In [24]:
#Fine-Tune Clusters (Best for Precision)
# After removing Cluster 1, re-scale the filtered data
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)  # Now lengths match

# Check new clusters
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=['Recency', 'Frequency', 'Monetary']
)
print(cluster_centers)

     Recency   Frequency      Monetary
0  11.844365   24.622987  8.588568e+04
1   9.300000  364.828571  1.193500e+06
2  60.934844    7.308074  3.807489e+04
3  19.750000   97.000000  8.334898e+06


In [25]:
rfm = rfm[~rfm['Cluster'].isin([1, 3])]  # Keep only clusters 0 and 2

In [28]:
# 1. Filter clusters (create explicit copy)
rfm_clean = rfm[~rfm['Cluster'].isin([1, 3])].copy()

# 2. Assign high-risk label
rfm_clean['is_high_risk'] = (rfm_clean['Cluster'] == 2).astype(int)

# 3. Verify
print("Risk distribution:")
print(rfm_clean['is_high_risk'].value_counts(normalize=True))

# 4. Merge back (using the cleaned version)
df = pd.merge(df, rfm_clean[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
df['is_high_risk'] = df['is_high_risk'].fillna(0)

Risk distribution:
is_high_risk
0    0.612939
1    0.387061
Name: proportion, dtype: float64


In [31]:
# Only label top 25% of Recency as high-risk
recency_threshold = rfm_clean['Recency'].quantile(0.75)
rfm_clean['is_high_risk'] = (rfm_clean['Recency'] >= recency_threshold).astype(int)

print("Adjusted Risk Distribution:")
print(rfm_clean['is_high_risk'].value_counts(normalize=True))

Adjusted Risk Distribution:
is_high_risk
0    0.739857
1    0.260143
Name: proportion, dtype: float64


In [32]:
# 1. First verify rfm_clean contains the column
print("Columns in rfm_clean:", rfm_clean.columns.tolist())

# 2. Check CustomerId data types
print("df CustomerId dtype:", df['CustomerId'].dtype)
print("rfm_clean CustomerId dtype:", rfm_clean['CustomerId'].dtype)

# 3. Check merge keys exist
print("CustomerIds in df:", df['CustomerId'].nunique())
print("CustomerIds in rfm_clean:", rfm_clean['CustomerId'].nunique())

# 4. Alternative merge (more robust)
df = df.merge(
    rfm_clean[['CustomerId', 'is_high_risk']],
    on='CustomerId',
    how='left'
)

# 5. Safe fillna
if 'is_high_risk' not in df.columns:
    df['is_high_risk'] = 0  # Initialize if merge failed
else:
    df['is_high_risk'] = df['is_high_risk'].fillna(0)

# 6. Final verification
print("\nMerge verification:")
print("Columns in df:", df.columns.tolist())
print("Risk distribution:")
print(df['is_high_risk'].value_counts(normalize=True))

Columns in rfm_clean: ['CustomerId', 'Recency', 'Frequency', 'Monetary', 'Cluster', 'is_high_risk']
df CustomerId dtype: object
rfm_clean CustomerId dtype: object
CustomerIds in df: 3712
CustomerIds in rfm_clean: 3648

Merge verification:
Columns in df: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'Outlier_Amount_IQR', 'Outlier_Value_IQR', 'Z_Amount', 'Z_Value', 'Outlier_Amount_Z', 'Outlier_Value_Z', 'is_high_risk_x', 'is_high_risk_y', 'is_high_risk']
Risk distribution:
is_high_risk
0.0    0.927845
1.0    0.072155
Name: proportion, dtype: float64


In [33]:
# Remove duplicate columns (keep only the final 'is_high_risk')
cols_to_drop = [col for col in df.columns if col.startswith('is_high_risk_')]
df = df.drop(columns=cols_to_drop)

# Verify cleanup
print("Cleaned columns:", [c for c in df.columns if 'high_risk' in c])

Cleaned columns: ['is_high_risk']


In [34]:
high_risk_customers = df[df['is_high_risk'] == 1]['CustomerId']
print(rfm_clean[rfm_clean['CustomerId'].isin(high_risk_customers)].describe())

          Recency   Frequency      Monetary     Cluster  is_high_risk
count  948.000000  948.000000  9.480000e+02  948.000000         948.0
mean    69.155063    7.256329  4.273887e+04    1.410338           1.0
std     11.689136   15.560513  1.299652e+05    0.819108           0.0
min     53.000000    1.000000 -9.420000e+04    1.000000           1.0
25%     59.000000    1.000000  1.000000e+03    1.000000           1.0
50%     68.000000    3.000000  9.000000e+03    1.000000           1.0
75%     81.000000    7.000000  2.892500e+04    1.000000           1.0
max     90.000000  181.000000  2.017630e+06    4.000000           1.0


In [35]:
unclustered = df[~df['CustomerId'].isin(rfm_clean['CustomerId'])]
print(f"{len(unclustered)} customers not clustered")

29984 customers not clustered


In [38]:
# Step 1: Label unclustered customers as low-risk
df.loc[~df['CustomerId'].isin(rfm_clean['CustomerId']), 'is_high_risk'] = 0

# Step 2: Recalculate risk distribution
print("Final Risk Distribution:")
print(df['is_high_risk'].value_counts(normalize=True))

# Step 3: Save for modeling
df.to_csv(r'C:\Users\ayedr\week-5-credit-risk-model\data\processed\final_labeled_data.csv', index=False)

Final Risk Distribution:
is_high_risk
0.0    0.927845
1.0    0.072155
Name: proportion, dtype: float64


In [40]:
print("Columns in main DataFrame:", df.columns.tolist())

Columns in main DataFrame: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'Outlier_Amount_IQR', 'Outlier_Value_IQR', 'Z_Amount', 'Z_Value', 'Outlier_Amount_Z', 'Outlier_Value_Z', 'is_high_risk']


In [41]:
# Merge RFM metrics into main DataFrame (optional)
df = df.merge(
    rfm_clean[['CustomerId', 'Recency', 'Frequency', 'Monetary']],
    on='CustomerId',
    how='left'
)

# Now you can analyze directly
print(df[df['is_high_risk'] == 1][['Recency', 'Frequency', 'Monetary']].describe())

           Recency    Frequency      Monetary
count  6879.000000  6879.000000  6.879000e+03
mean     65.923681    40.589184  1.303425e+05
std      10.980321    52.465579  2.127384e+05
min      53.000000     1.000000 -9.420000e+04
25%      56.000000     6.000000  1.393000e+04
50%      63.000000    16.000000  4.430000e+04
75%      74.000000    57.000000  1.536115e+05
max      90.000000   181.000000  2.017630e+06


In [43]:
#Feature Selection

# Include RFM features + original features
features = [
    'Recency', 'Frequency', 'Monetary',  # RFM
    'Amount', 'Value', 'PricingStrategy',  # Transaction features
    'ProductCategory', 'ChannelId'        # Categorical features
]

X = df[features]
y = df['is_high_risk']
# Convert categoricals
X = pd.get_dummies(X, columns=['ProductCategory', 'ChannelId'])


In [45]:
print("Missing values per column:")
print(X.isna().sum())

Missing values per column:
Recency                               29984
Frequency                             29984
Monetary                              29984
Amount                                    0
Value                                     0
PricingStrategy                           0
ProductCategory_airtime                   0
ProductCategory_data_bundles              0
ProductCategory_financial_services        0
ProductCategory_movies                    0
ProductCategory_other                     0
ProductCategory_ticket                    0
ProductCategory_transport                 0
ProductCategory_tv                        0
ProductCategory_utility_bill              0
ChannelId_ChannelId_1                     0
ChannelId_ChannelId_2                     0
ChannelId_ChannelId_3                     0
ChannelId_ChannelId_5                     0
dtype: int64


In [48]:
# Calculate median values from clustered customers
median_recency = rfm_clean['Recency'].median()
median_frequency = rfm_clean['Frequency'].median()
median_monetary = rfm_clean['Monetary'].median()

# Fill missing values
X['Recency'] = X['Recency'].fillna(median_recency)
X['Frequency'] = X['Frequency'].fillna(median_frequency)
X['Monetary'] = X['Monetary'].fillna(median_monetary)
print("Missing values after imputation:")
print(X.isna().sum())

Missing values after imputation:
Recency                               0
Frequency                             0
Monetary                              0
Amount                                0
Value                                 0
PricingStrategy                       0
ProductCategory_airtime               0
ProductCategory_data_bundles          0
ProductCategory_financial_services    0
ProductCategory_movies                0
ProductCategory_other                 0
ProductCategory_ticket                0
ProductCategory_transport             0
ProductCategory_tv                    0
ProductCategory_utility_bill          0
ChannelId_ChannelId_1                 0
ChannelId_ChannelId_2                 0
ChannelId_ChannelId_3                 0
ChannelId_ChannelId_5                 0
dtype: int64


In [47]:
print("Missing values after imputation:")
print(X.isna().sum())

Missing values after imputation:
Recency                               0
Frequency                             0
Monetary                              0
Amount                                0
Value                                 0
PricingStrategy                       0
ProductCategory_airtime               0
ProductCategory_data_bundles          0
ProductCategory_financial_services    0
ProductCategory_movies                0
ProductCategory_other                 0
ProductCategory_ticket                0
ProductCategory_transport             0
ProductCategory_tv                    0
ProductCategory_utility_bill          0
ChannelId_ChannelId_1                 0
ChannelId_ChannelId_2                 0
ChannelId_ChannelId_3                 0
ChannelId_ChannelId_5                 0
dtype: int64


In [49]:
print("Original class distribution:")
print(y.value_counts(normalize=True))

Original class distribution:
is_high_risk
0.0    0.927845
1.0    0.072155
Name: proportion, dtype: float64


In [50]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

print("\nBalanced class distribution after SMOTE:")
print(pd.Series(y_res).value_counts())


Balanced class distribution after SMOTE:
is_high_risk
0.0    88457
1.0    88457
Name: count, dtype: int64


In [51]:
#Verify All Steps Are Completed:

# Confirm final DataFrame structure
print("Final columns:", df.columns.tolist())
print("Risk distribution:\n", df['is_high_risk'].value_counts(normalize=True))


Final columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'Outlier_Amount_IQR', 'Outlier_Value_IQR', 'Z_Amount', 'Z_Value', 'Outlier_Amount_Z', 'Outlier_Value_Z', 'is_high_risk', 'Recency', 'Frequency', 'Monetary']
Risk distribution:
 is_high_risk
0.0    0.927845
1.0    0.072155
Name: proportion, dtype: float64


In [52]:
df.to_csv(r'C:\Users\ayedr\week-5-credit-risk-model\data\processed\features_with_risk.csv', index=False)

## Task 4 Completion

### High-Risk Customer Definition
- Cluster 1 (of 3) identified as high-risk based on:
  - Recency > 60 days
  - Frequency < 10 transactions
  - Monetary value in bottom quartile

### Class Distribution
- Original: 7.2% high-risk
- After SMOTE: 50% high-risk (for modeling)

### Key Decisions
- Negative Monetary values treated as refunds
- Unclustered customers labeled as low-risk
- RFM features standardized before clustering